# Working Azure Translator Example

The Azure Translator Text SDK (`azure-ai-translation-text`) is still in preview and has authentication issues. The REST API approach below works reliably.

## Key differences:
- **SDK approach**: Uses `azure-ai-translation-text` package (has auth issues)
- **REST API approach**: Direct HTTP calls to the Translator API (works reliably)

The REST API approach uses:
- `Ocp-Apim-Subscription-Key` header for your subscription key
- `Ocp-Apim-Subscription-Region` header for your region
- Direct HTTP POST to the translate endpoint

# Resources
https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/translation/azure-ai-translation-text/README.md

This attempt failed because the API is in preview and we could not find a working example. We were getting authenication issues despite using the correct API key. So instead with translator its recommended to use the WebAPI code example.

In [ ]:
%pip install azure-ai-translation-text==1.0.0b1

In [20]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.translation.text import TextTranslationClient
from azure.core.exceptions import HttpResponseError

In [ ]:
apikey = ""
region = "westus"
endpoint = "https://api.cognitive.microsofttranslator.com/"

In [22]:
credential = AzureKeyCredential(apikey)
text_translator = TextTranslationClient(
    credential=credential,
    endpoint=endpoint,
    region=region
)

In [ ]:
try:
    # Use correct language codes (ja for Japanese, not jp)
    langs = ["ja", "es", "de"]
    input_text = ["This is a test"]

    response = text_translator.translate(content=input_text, to=langs)
    translation = response[0] if response else None

    if translation:
        detected_language = translation.detected_language
        if detected_language:
            print(
                f"Detected languages of the input text: {detected_language.language} with score: {detected_language.score}."
            )
        for translated_text in translation.translations:
            print(f"Text was translated to: '{translated_text.to}' and the result is: '{translated_text.text}'.")

except HttpResponseError as exception:
    print(f"HTTP Error occurred: {exception}")
    if hasattr(exception, 'error') and exception.error is not None:
        print(f"Error Code: {exception.error.code}")
        print(f"Message: {exception.error.message}")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Working REST API approach for Azure Translator
import requests
import json

# Your Azure Translator credentials
subscription_key = "3RsadaEHwTb9YHekAPHuBB6ra9Ndl5Kzlr5nXMDiOayJ5OKLiiB0JQQJ99BFAC4f1cMXJ3w3AAAbACOGcJp3"
region = "westus"
endpoint = "https://api.cognitive.microsofttranslator.com"

# Text to translate
text_to_translate = ["This is a test"]
target_languages = ["ja", "es", "de"]

# Construct the request
path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'to': target_languages
}

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Ocp-Apim-Subscription-Region': region,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# Request body
body = [{'text': text} for text in text_to_translate]

try:
    import uuid
    
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    
    if request.status_code == 200:
        print("Translation successful!")
        for i, translation_result in enumerate(response):
            print(f"\nOriginal text: {text_to_translate[i]}")
            
            # Detected language
            if 'detectedLanguage' in translation_result:
                detected = translation_result['detectedLanguage']
                print(f"Detected language: {detected['language']} (confidence: {detected['score']})")
            
            # Translations
            for translation in translation_result['translations']:
                print(f"Translation to {translation['to']}: {translation['text']}")
    else:
        print(f"Error: {request.status_code}")
        print(f"Response: {response}")
        
except Exception as e:
    print(f"An error occurred: {e}")